In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *
from keras.backend.tensorflow_backend import set_session
gpu_no = '0' # or '1'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))
#最后加入了动态分配显存之后得到对应的结果

In [15]:
tokenizer = BertTokenizer.from_pretrained('/home/huarui/competitiondata/google-qa-labeling/bert-base-uncased-vocab.txt')
#通过词典导入对应的分词器
MAX_SEQUENCE_LENGTH = 16

df_train = pd.read_csv('/home/huarui/competitiondata/google-qa-labeling/train.csv')
df_test = pd.read_csv('/home/huarui/competitiondata/google-qa-labeling/test.csv')
df_sub = pd.read_csv('/home/huarui/competitiondata/google-qa-labeling/sample_submission.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
#output_categories为后续的需要输出分类概率的对象
input_categories = list(df_train.columns[[1,2,5]])
#取出对应的第一个元素，第二个元素以及第五个元素
#输入的问题input_categories之中只有['question_title','question_body','answer']
#与问题和答案相关，所以只需要截取这三项即可
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']


取出的内容之后df_train.columns[11:],从下标11之后的内容为需要进行分配的内容，所以取出[11:]进行操作

In [16]:
print(df_train.columns)

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

In [17]:
def _convert_to_transformer_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    print('_convert_to_transformer_inputs')
    #使用_convert_to_transformer_inputs的时候，下面对应的return_id
    #相应的函数暂时不会被调用
    def return_id(str1, str2, truncation_strategy, length):
        print('return_id')
        #truncation_strategy = 'longest_first'
        #length = 16,truncation = True
        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy,truncation=True)
        #之前定义过tokenizer = BertTokenizer.from_pretrained
        #('/home/huarui/competitiondata/google-qa-labeling/bert-base-uncased-vocab.txt')
        #这里使用bert-base-uncased-vocab.txt对相应的语句进行分词标记
        input_ids =  inputs["input_ids"]
        #input_ids = 
        #[101, 2054, 2572, 1045, 3974, 2043, 2478, 5331, 
        #10868, 2612, 1997, 1037, 26632, 10014, 1029, 102]
        
        #这里面的对应编码直接使用bert-base-uncased-vocab.txt
        #里面对应的编码的内容进行操作
        input_masks = [1] * len(input_ids)
        #input_masks = [1,1,1,1,1,1
        #1,1,1,1,1,1,1,1,1,1]
        input_segments = inputs["token_type_ids"]
        #input_segments = [0,0,0,0,0,0
        #0,0,0,0,0,0,0,0,0,0]
        padding_length = length - len(input_ids)
        #padding_length = 0(注意这里定义的length=16，
        #如果相应的长度没有到达16的时候会在后面填充
        #相应的0数值)
        padding_id = tokenizer.pad_token_id
        #padding_id = 0
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        #input_ids = [101, 2054, 2572, 1045, 3974, 2043, 2478, 5331, 10868, 2612, 1997, 1037, 26632, 10014, 1029, 102]
        #input_masks = [1 ,1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        #input_segments = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        return [input_ids, input_masks, input_segments]
    
    print('process1')
    input_ids_q, input_masks_q, input_segments_q = return_id(
        title + ' ' + question, None, 'longest_first', max_sequence_length)
    #这里面调用对应的嵌套函数return_id的内容
    print('process2')
    input_ids_a, input_masks_a, input_segments_a = return_id(
        answer, None, 'longest_first', max_sequence_length)
    print('finish')
    return [input_ids_q, input_masks_q, input_segments_q,
            input_ids_a, input_masks_a, input_segments_a]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    print('compute_input_arrays')
    input_ids_q, input_masks_q, input_segments_q = [], [], []
    input_ids_a, input_masks_a, input_segments_a = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer
        #t的内容为对应的question_title
        #q的内容为对应的question_body
        #a的内容为对应的answer
        ids_q, masks_q, segments_q, ids_a, masks_a, segments_a = \
        _convert_to_transformer_inputs(t, q, a, tokenizer, max_sequence_length)
        #这里具体操作的时候会调用里面的内部嵌套函数return_id函数
        input_ids_q.append(ids_q)
        input_masks_q.append(masks_q)
        input_segments_q.append(segments_q)

        input_ids_a.append(ids_a)
        input_masks_a.append(masks_a)
        input_segments_a.append(segments_a)
    
    print('finish')
    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(input_masks_q, dtype=np.int32), 
            np.asarray(input_segments_q, dtype=np.int32),
            np.asarray(input_ids_a, dtype=np.int32), 
            np.asarray(input_masks_a, dtype=np.int32), 
            np.asarray(input_segments_a, dtype=np.int32)]

def compute_output_arrays(df, columns):
    print('compute_output_arrays')
    return np.asarray(df[columns])

~~~inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy,truncation=True)
~~~
这里面对应的str1 =  What am I losing when using extension tubes instead of a macro lens? After playing around with macro photography on-the-cheap (read: reversed lens, rev. lens mounted on a straight lens, passive extension tubes), I would like to get further with this. The problems with the techniques I used is that focus is manual and aperture control is problematic at best. This limited my setup to still subjects (read: dead insects) Now, as spring is approaching, I want to be able to shoot live insects. I believe that for this, autofocus and settable aperture will be of great help.

So, one obvious but expensive option is a macro lens (say, EF 100mm Macro) However, I am not really interested in yet another prime lens. An alternative is the electrical extension tubes.

Except for maximum focusing distance, what am I losing when using tubes (coupled with a fine lens, say EF70-200/2.8) instead of a macro lens?
str2 = None,
truncation_strategy = 'longest_first',max_length = 16,tokenization为之前的tokenizer = BertTokenizer.from_pretrained('/home/huarui/competitiondata/google-qa-labeling/bert-base-uncased-vocab.txt')预训练结果的内容，
inputs = @@@
{'input_ids': [101, 2054, 2572, 1045, 3974, 2043, 2478, 5331, 10868, 2612, 1997, 1037, 26632, 10014, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
bert的调用分两步，第一步先把文本转化为BERT模型输入的形式(一个三个参数input_ids,input_mask,token_type_ids)，然后调用模型使用即可得到文本的向量表示
**这里可以查看一下bert_config对应的定义**
from bert import modeling    
model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings
)

In [18]:
def compute_spearmanr_ignore_nan(trues, preds):
#trues,preds对应的内容为valid_outputs,valid_preds[-1]
#因为相应的数组内容均为小数，所以这里使用对应的斯皮尔曼相关系数
#进行操作
    rhos = []
    print('compute_spearmanr_ignore_nan')
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
    #计算相应的内容的时候，需要先将对应的矩阵转置过来
        print('tcol = ***')
        print(tcol)
        print('pcol = ***')
        print(pcol)
        rhos.append(spearmanr(tcol, pcol).correlation)
    #spearmanr为上面定义的相应的函数
    #这里spearmanr为对应的斯皮尔曼等级相关，
    #斯皮尔曼等级相关系数是反映两组变量之间联系的密切程度，
    #它和相关系数r一样，取值在-1到+1之间，所不同的是它是
    #建立在等级的基础上计算的。
    return np.nanmean(rhos)

def create_model():
    #!!!这里面定义了内部的每一层，还并没有进行相应的乘除操作内容!!!
    print('create_model')
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    print(q_id.shape)
    print(a_id.shape)
    #q_id = (None,16),a_id = (None,16)
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    #q_mask = (None,16),a_id = (None,16)
    print(q_mask.shape)
    print(a_mask.shape)
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    #q_atn = (None,16),a_atn = (None,16)
    print(q_atn.shape)
    print(a_atn.shape)
    #!!!上面的操作指定q_id,a_id,q_mask,a_mask,q_atn,a_atn都是对应的
    #输入部分的相应的内容,使用对应的tf.keras.layers.Input进行相应内
    #容的指定
    #model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, a_id, a_mask, a_atn,], outputs=x)
    #上面对应的inputs内容指从inputs数组之中得到的相应的inputs的对应内容
    
    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    # normally ".from_pretrained('bert-base-uncased')", but because of no internet, the 
    # pretrained model has been downloaded manually and uploaded to kaggle. 
    bert_model = TFBertModel.from_pretrained(
        '/home/huarui/competitiondata/google-qa-labeling/bert-base-uncased-tf_model.h5', config=config)
    #!!!之前文件之中的内容为Unable to open file(unable to lock file,errno=11,error message = 'Resource temporarily unavailable')
    #!!!因为另外一个程序打开了相应的bert-base-uncased-tf_model.h5文件，不能多个程序同时打开一个文件
    print('bert_model = ***')
    print(bert_model)
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    a_embedding = bert_model(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    #使用embedding层的时候，对应的三个相应的参数都需要定义
    
    #q_id = (None,16),q_mask = (None,16),q_atn = (None,16),
    #a_id = (None,16),a_mask = (None,16),a_atn = (None,16)
    #q_embedding = (None,16,768)
    print('q_embedding = ***')
    print(q_embedding)
    #a_embedding = (None,16,768)
    print('a_embedding = ***')
    print(a_embedding)
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    #对时域1D的区域进行平均池化
    a = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    #q = (None,768)，这个768为自己定义的相应的神经网络层的对应内容
    #a = (None,768)
    print('q = ***')
    print(q.shape)
    print('a = ***')
    print(a.shape)
    x = tf.keras.layers.Concatenate()([q, a])
    #q = (None,768),a = (None,768),Concatenate()对
    #两个数组后面的维度进行拼接
    #x1 = (None,1536)
    print('x1 = ***')
    print(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    #1.随机临时删掉网络中的部分0.2神经元，输入输出保持不变
    #2.把输入x通过修改后的网络前向传播，把得到的损失结果通过
    #修改的网络进行反向传播
    #3.对一小批训练样本，先前向传播然后反向传播损失并根据
    #随机梯度下降法更新参数(w,b)
    #x2 = (None,1536)
    #第一个参数对应着相应的batch内容
    print('x2 = ***')
    print(x)
    x = tf.keras.layers.Dense(30, activation='sigmoid')(x)
    #x3 = (None,30)
    print('x3 = ***')
    print(x)
    print('model = ***')
    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, a_id, a_mask, a_atn,], outputs=x)
    #由开头的inputs构成的六个对应内容的输入到相应内容的输出进行model变换
    #这里面的q_id,q_mask,q_atn,a_id,a_mask,a_atn构成相应的inputs的内容
    #与上面需要进行操作的相应的内容相同
    print(model)
    return model

In [19]:
outputs = compute_output_arrays(df_train, output_categories)
#之前的output_categories = list(df_train.columns[11:])
#对应内容为从下标11之后的对应的值的内容
print('outputs = ###')
print(outputs.shape)
#outputs为df_train在之后的output_categories之中得到的结果矩阵
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
#inputs里面包含[input_ids_q,input_masks_q,input_segments_q,input_ids_a,
#input_masks_a,input_segments_a]
#其中input_ids_q当中包含所有的疑问句的ids,input_masks_q包含所有的masks中的input_segments_q
#input_segments_q当中包含所有的疑问句的input_segments_a...
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

compute_output_arrays
outputs = ###
(6079, 30)
compute_input_arrays


_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_

process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
ret

finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_conv

finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_conv

...

_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_

_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_

_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_

process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
ret

_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_transformer_inputs
process1
return_id
process2
return_id
finish
_convert_to_

In [20]:
df_train.question_body

0       After playing around with macro photography on...
1       I am trying to understand what kinds of places...
2       I'm working on a PCB that has through-hole com...
3       An affidavit, from what i understand, is basic...
4       I am trying to make a binary image. I want mor...
                              ...                        
6074    I am curious if anyone uses a skiing helmet fo...
6075    I have a road bike with a front brake that wea...
6076    I'm tailing a log file using tail -f messages....
6077    What are people's views on this?  To be specif...
6078    Newbie question.\n\nWhy is it that there's a b...
Name: question_body, Length: 6079, dtype: object

首先使用outputs = compute_output_arrays(df_train, output_categories)取出[11:]后面11项指标当中对应的概率
接着使用inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
先使用 for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer
取出其中的t，q，a，其中t为对应的问题标题，q为问题内容，answer为问题的回答


In [21]:
df_sub
#这个时候的df_sub还是预测之前的对应值

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,9569,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,...,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159
472,9590,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,...,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355
473,9597,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,...,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467
474,9623,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,...,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720


In [22]:
gkf = GroupKFold(n_splits=5).split(X=df_train.question_body, groups=df_train.question_body)
#df_train.question_body为获取相应的问题的内容信息
valid_preds = []
test_preds = []
for fold, (train_idx, valid_idx) in enumerate(gkf):
    print('fold = %d'%fold)
    print('train_idx = ***')
    print(train_idx)
    #[1,3,5,...6076,6077,6078]
    print('valid_idx = ***')
    print(valid_idx)
    #[0,2,4,...6073,6074,6075]
    # will actually only do 2 folds (out of 5) to manage < 2h
    print('!!!before  df_sub = !!!')
    print(df_sub)
    if fold in [0,2]:
    #这里的fold对应的值为0，总共有[0,1,2,3,4]共5折的对应的内容
    #train_idx = [1,3,5,...6076,6077,6078]
    #valid_idx = [0,2,4,...6073,6074,6075]
        train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
        train_outputs = outputs[train_idx]
    #train_inputs将inputs中train_idx中的对应内容提取出来
    #train_outputs将对应的输出权重提取出来
        valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
        valid_outputs = outputs[valid_idx]
    #valid_inputs将inputs中的valid_idx内容提取出来
    #valid_outputs将outputs中的valid_idx内容提取出来
    
    #这个K前面的内容有定义
    #import tensorflow.keras.backend as K
        K.clear_session()
        #The keras documentation says regarding clear_session:Destroys 
        #the current TF graph and creates a new one.Useful to avoid
        #clutter from old models/layers.
        
        #当你在进行keras的交叉验证的时候，例如用5折，对于fold_0,fold_1,
        #一直到fold_4,都应该有一个独立的模型，所以在每折的开头都需要加上
        #clear_session(),否则上一折的训练集成了这一折的验证集，数据泄露
        
        #不清空的话，那么graph上的node越来越多，内存问题，时间问题越来越
        #严重
        print('1  !!!df_sub!!!')
        print(df_sub)
        print('create_model')
        model = create_model()
        print('2  !!!df_sub!!!')
        print(df_sub)
        optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
        model.compile(loss='binary_crossentropy', optimizer=optimizer)
        model.fit(train_inputs, train_outputs, epochs=3, batch_size=6)
        #train_inputs与train_outputs里面的内容相似，修改对应的epochs
        #!!!tensorflow里面不会显示神经网络层之中的运转结果，与pytorch
        #不同，所以tensorflow之中当model.fit()函数调用的时候
        valid_preds.append(model.predict(valid_inputs))
        test_preds.append(model.predict(test_inputs))
        print('3  !!!df_sub!!!')
        print(df_sub)
        rho_val = compute_spearmanr_ignore_nan(valid_outputs,valid_preds[-1])
        print('validation score = ', rho_val)
        #通过观察两个数组相应的斯皮尔曼相关系数来判断对应的内容
        #有无上升变化
        break

fold = 0
train_idx = ***
[   1    3    5 ... 6076 6077 6078]
valid_idx = ***
[   0    2    4 ... 6073 6074 6075]
!!!before  df_sub = !!!
     qa_id  question_asker_intent_understanding  question_body_critical  \
0       39                              0.00308                 0.00308   
1       46                              0.00448                 0.00448   
2       70                              0.00673                 0.00673   
3      132                              0.01401                 0.01401   
4      200                              0.02074                 0.02074   
..     ...                                  ...                     ...   
471   9569                              0.99159                 0.99159   
472   9590                              0.99355                 0.99355   
473   9597                              0.99467                 0.99467   
474   9623                              0.99720                 0.99720   
475   9640                            

Some weights of the model checkpoint at /home/huarui/competitiondata/google-qa-labeling/bert-base-uncased-tf_model.h5 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at /home/huarui/competitiondata/google-qa-labeling/bert-base-uncased-tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


bert_model = ***
q_embedding = ***
Tensor("tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0", shape=(None, 16, 768), dtype=float32)
a_embedding = ***
Tensor("tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm_1/add_1:0", shape=(None, 16, 768), dtype=float32)
q = ***
(None, 768)
a = ***
(None, 768)
x1 = ***
Tensor("concatenate/concat:0", shape=(None, 1536), dtype=float32)
x2 = ***
Tensor("dropout_37/cond/Identity:0", shape=(None, 1536), dtype=float32)
x3 = ***
Tensor("dense/Sigmoid:0", shape=(None, 30), dtype=float32)
model = ***
2  !!!df_sub!!!
     qa_id  question_asker_intent_understanding  question_body_critical  \
0       39                              0.00308                 0.00308   
1       46                              0.00448                 0.00448   
2       70                              0.00673                 0.00673   
3      132                              0.01401                 0.01401   
4      200                          

W1204 04:48:12.026117 140666326013760 optimizer_v2.py:1275] Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.
W1204 04:48:12.029128 140666326013760 optimizer_v2.py:1275] Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.
W1204 04:48:30.659389 140666326013760 optimizer_v2.py:1275] Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.
W1204 04:48:30.662398 140666326013760 optimizer_v2.py:1275] Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.


811/811 [==============================] - 118s 145ms/step - loss: 0.4083
Epoch 2/3
811/811 [==============================] - 118s 145ms/step - loss: 0.3799
Epoch 3/3
811/811 [==============================] - 118s 145ms/step - loss: 0.3619
3  !!!df_sub!!!
     qa_id  question_asker_intent_understanding  question_body_critical  \
0       39                              0.00308                 0.00308   
1       46                              0.00448                 0.00448   
2       70                              0.00673                 0.00673   
3      132                              0.01401                 0.01401   
4      200                              0.02074                 0.02074   
..     ...                                  ...                     ...   
471   9569                              0.99159                 0.99159   
472   9590                              0.99355                 0.99355   
473   9597                              0.99467                 0.9

In [23]:
df_sub
#df_sub里面对应的内容为sample_submission.csv中读取到的相应的内容

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,9569,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,...,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159,0.99159
472,9590,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,...,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355,0.99355
473,9597,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,...,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467,0.99467
474,9623,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,...,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720,0.99720


In [24]:
test_preds

[array([[0.95972437, 0.6816642 , 0.26129523, ..., 0.02528952, 0.96068054,
         0.94593996],
        [0.8462833 , 0.41339007, 0.00639458, ..., 0.11669905, 0.12870048,
         0.86298496],
        [0.945806  , 0.7287406 , 0.03877641, ..., 0.05309787, 0.89714664,
         0.92950016],
        ...,
        [0.8891869 , 0.45125693, 0.01177636, ..., 0.18139923, 0.61951274,
         0.8859671 ],
        [0.91733   , 0.6624307 , 0.03543678, ..., 0.0419499 , 0.806015  ,
         0.9157155 ],
        [0.9217964 , 0.6376776 , 0.01346106, ..., 0.08256797, 0.10628865,
         0.9064478 ]], dtype=float32)]

In [25]:
df_sub.iloc[:, 1:] = np.average(test_preds, axis=0) # for weighted average set weights=[...]
#iloc[:,:] 行列切片以“，”隔开，前面的冒号就是取行数，后面的冒号是取列数
#这一部分运行完之后df_sub发生相应的变化，
#df_sub.iloc[:,1:]行的位置为:,即所有行对应的内容都不发生变化
#列对应的位置为[1:],即列取test_preds对应的所有的值
#axis = 0为表示纵轴，进行操作的方向为从上到下
df_sub.to_csv('submission.csv', index=False)

In [26]:
df_sub

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.959724,0.681664,0.261295,0.760404,0.511794,0.621115,0.704853,0.679393,0.385776,...,0.872446,0.955890,0.648850,0.975013,0.969411,0.925423,0.028893,0.025290,0.960681,0.945940
1,46,0.846283,0.413390,0.006395,0.658363,0.797794,0.910795,0.581826,0.424709,0.147360,...,0.703284,0.952977,0.624299,0.963323,0.981471,0.897790,0.884592,0.116699,0.128700,0.862985
2,70,0.945806,0.728741,0.038776,0.828527,0.871823,0.954775,0.688364,0.452183,0.386949,...,0.886671,0.965040,0.592158,0.981478,0.984580,0.923770,0.119436,0.053098,0.897147,0.929500
3,132,0.878449,0.373438,0.006043,0.810668,0.843688,0.954103,0.550669,0.447762,0.275598,...,0.673965,0.960154,0.767095,0.977669,0.978693,0.838932,0.701834,0.068670,0.888365,0.893946
4,200,0.890775,0.466216,0.005460,0.885543,0.926870,0.925903,0.728777,0.717770,0.147870,...,0.776592,0.917683,0.605983,0.978814,0.976265,0.826033,0.227865,0.138893,0.512716,0.901170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,9569,0.863475,0.419606,0.003370,0.753723,0.872975,0.963403,0.599065,0.456660,0.107117,...,0.679574,0.969036,0.717359,0.980769,0.988401,0.904827,0.951524,0.137169,0.111779,0.911702
472,9590,0.794946,0.377680,0.021124,0.765196,0.938919,0.876637,0.538661,0.512182,0.111684,...,0.701154,0.915014,0.659960,0.963237,0.967811,0.814277,0.319291,0.082002,0.690716,0.913628
473,9597,0.889187,0.451257,0.011776,0.710874,0.835076,0.833182,0.525833,0.518960,0.211071,...,0.721927,0.948569,0.720793,0.971103,0.974728,0.790350,0.421831,0.181399,0.619513,0.885967
474,9623,0.917330,0.662431,0.035437,0.898578,0.848180,0.953515,0.611709,0.546330,0.250693,...,0.891598,0.932975,0.743412,0.964432,0.986506,0.908917,0.293835,0.041950,0.806015,0.915716
